### Import libraries

In [93]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
pd.options.plotting.backend = "plotly"
import math

In [94]:
from utils import SimpleExponentialSmoothing, plot_ts_forecast, build_forecast, qualityMAPE

### Load data

In [95]:
def load_data(ticker = 'TRNFP'):
  ts = pd.read_csv(f'{ticker}_moex_10m_data.csv')

  ts['Datetime'] = pd.to_datetime(ts['Datetime'])

  ts.set_index('Datetime', inplace = True)

  ts = ts['Close'].resample("D").sum()

  ts = ts[ts > 0.0]

  ts = pd.DataFrame(ts)

  return ts

In [96]:
ts = load_data()

### Look at data

In [97]:
ts

,Close
Datetime,
2021-10-01,115365.0
2021-10-04,212142.5
2021-10-05,313241.0
2021-10-06,405293.5
2021-10-07,492914.0
...,...
2024-10-29,523452.5
2024-10-30,524907.5
2024-10-31,515077.5


In [98]:
ts.plot().update_layout(height=350, width=1300).show()

### Make prediction

In [99]:
row_num = ts.shape[0]
boundaries = []

for i in range(0, row_num, round(row_num/4)):
  boundaries.append(ts.index[i])

boundaries

[Timestamp('2021-10-01 00:00:00'),
 Timestamp('2022-08-03 00:00:00'),
 Timestamp('2023-05-04 00:00:00'),
 Timestamp('2024-02-01 00:00:00')]

In [100]:
ALPHA = [0.9, 0.7, 0.4, 0.2, .15, 0.1, 0.05, 0.01, 0.001, 0.00001]
ESparams = [{'alpha':alpha} for alpha in ALPHA]
FRC_ts = build_forecast(h=1, ts=ts.loc[boundaries[2]:boundaries[3]], alg_name = 'SimpleExponentialSmoothing', alg_title = 'ES', params = ESparams)

In [101]:
qlt_ses = pd.DataFrame(index = ts.columns, columns = FRC_ts.keys())

ix = ts.loc[boundaries[2]:boundaries[3]].index
for param_cntr in sorted(qlt_ses.columns):
    frc_ts = FRC_ts[param_cntr]
    qlt_ses[param_cntr],_ = qualityMAPE(ts.loc[ix], frc_ts.loc[ix])

qlt_ses[qlt_ses.columns].mean().sort_values()

,0
ES {'alpha': 0.9},0.011184
ES {'alpha': 0.7},0.012091
ES {'alpha': 0.4},0.015787
ES {'alpha': 0.2},0.023460
ES {'alpha': 0.15},0.027911
ES {'alpha': 0.1},0.035666
ES {'alpha': 0.05},0.051994
ES {'alpha': 0.01},0.100197
ES {'alpha': 0.001},0.176125
ES {'alpha': 1e-05},0.190262


In [102]:
alg_name = qlt_ses[qlt_ses.columns].mean().sort_values().index[0]
# plot_ts_forecast(ts.loc[boundaries[2]:boundaries[3]], FRC_ts[alg_name].loc[boundaries[2]:boundaries[3]]
#                , ts_num=0, alg_title='ES alpha=0.9')

In [103]:
qualityMAPE(ts, FRC_ts[alg_name])[0]

,0
Close,0.011132


In [104]:
# forecast delay = 1
h = 1

# data frame to store forecasted values
frc_ts = pd.DataFrame(columns = ts.columns)

# ES params
Params ={'alpha':0.9}

# generate forecasts for each Item
for cntr in ts.columns:
    frc_ts[cntr] = SimpleExponentialSmoothing(ts[cntr], h, Params)

plot_ts_forecast(ts.loc['2021-10-01':'2024-09-02'], frc_ts.loc['2021-10-01':'2024-12-31'], ts_num=0, alg_title='ES alpha=0.9')